In [1]:
from ipynb.fs.full.Infer_propagation_path_functions import *

from pymongo import MongoClient

In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.get_database('tweets')

In [3]:
COLLECTION_NAME = 'data_true_3'
tweets = db[COLLECTION_NAME].find({})
followers = db['followers'].find({})

tweet_user_ids = []
for tweet in tweets:
    tweet_user_ids.append(tweet['user']['id'])

tweet_user_ids

[18927441,
 1353759332572164102,
 1304886452669550592,
 25830131,
 1045812443505733634,
 1090933584,
 1388772182256463872,
 832051937876992000,
 56267926,
 61107227,
 1317571158674386946,
 2967917568,
 799374276705079296,
 3019136073,
 4753775000,
 1343243919790895115,
 997570319710937088,
 759600332,
 1618523779,
 3070488763,
 701849752997122048,
 2391268620,
 439478873,
 21465838,
 1215356223777845254,
 890519900648878080,
 2867646556,
 741075194513494017,
 2832195284,
 1177377254273241089,
 1185841790475296768,
 498890444,
 313821873,
 1084838106619088898,
 243308689,
 1059817573741723648,
 1321431814729322498,
 951698105010864128,
 524595038,
 47014168,
 2421228369,
 548858806,
 725337372271878144,
 935032427151372288,
 28354150,
 837031504861663233,
 392411896,
 31328814,
 62429777,
 776479178,
 988662527369281536,
 2963452340,
 1300462667602296833,
 1556917836,
 2269644660,
 332924640,
 1112763887756234752,
 69356275,
 76868619,
 939728251756433408,
 290959178,
 16229037,
 480399

In [4]:
db[COLLECTION_NAME].count_documents({'retweeted_status.user.id': 18927441})

625

In [5]:
'''
has a root user 1
root user has followers 1
If a root user followers intersects with tweets, they are cascade depth one users 1
if a cascade depth one user followers intersects with tweets, they are cascade depth two users
...
if a cascade depth n-1 user followers intersects with tweets, they are cascade depth n users

Lets say parent user publish a tweet and the child re-tweet that exact tweet.
child timestamp must be after the parent's time stamp
If child has many parents, parent with the most recent time stamp will be the owner of the child tweet  
'''
tweets = db[COLLECTION_NAME].find({})
#get root tweet
root_tweet = set()
for tweet in tweets:
    try:
        root_tweet.add(tweet['retweeted_status']['user']['id'])
    except KeyError as e:
        continue
print(root_tweet)

tweets = db[COLLECTION_NAME].find({})

# Create root node user_id, created_at

if len(root_tweet) == 1:
    for tweet in tweets:
        try:
            user_id = tweet['retweeted_status']['user']['id']
            created_at = tweet['retweeted_status']['created_at']
        except KeyError as e:
            continue
            
    print(user_id, created_at)

    root = Node(name = user_id, created_at = created_at, difference_seconds =  0)
    print(root.name)

    follower = db['followers'].find_one({'user_id' : root.name})
    try:
        print(follower['user_id'])
    except:
        print('None, DB does not have the user: ', root.name)
        
    #get depth 1 users

    if follower['user_id'] == root.name:
        root_followers = follower['followers']           

    root_followers

    tweets = db[COLLECTION_NAME].find({})
    followers = db['followers'].find({})

    tweet_user_ids = []
    for tweet in tweets:
        tweet_user_ids.append(tweet['user']['id'])

    tweet_user_ids

    cascade_depth_one_user_ids = list(set(root_followers) & set(tweet_user_ids))

    print(len(cascade_depth_one_user_ids))

    tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 18927441})
    followers = db['followers'].find({})
    print('add depth 1 users to the tree')
    # add depth 1 users to the tree
    for user_id in cascade_depth_one_user_ids:
        tweets = db[COLLECTION_NAME].find({'retweeted_status.user.id': 18927441})
        print(tweets)
        for tweet in tweets:
            if tweet['user']['id'] == user_id:
                print(tweet['created_at'])
                difference = difference_datetime(tweet['created_at'], root.created_at).total_seconds()
                print(difference)
                globals()[f'user_{user_id}'] = Node(name = user_id, created_at = tweet['created_at'], difference_seconds = difference , parent = root)


    for pre, fill, node in RenderTree(root):
        print("%s user_id: %s, created_at: %s, difference_seconds: %s" % (pre, node.name, node.created_at, node.difference_seconds))
else:
    multiple_root_users(root_tweet)
#     raise('Cascade has more than one root user')

{18927441}
18927441 Thu May 13 16:31:02 +0000 2021
18927441
18927441
37
add depth 1 users to the tree
Thu May 13 16:34:43 +0000 2021
221.0
Thu May 13 20:25:25 +0000 2021
14063.0
Thu May 13 21:23:48 +0000 2021
17566.0
Thu May 13 18:09:55 +0000 2021
5933.0
Fri May 14 16:13:03 +0000 2021
85321.0
Thu May 13 23:46:54 +0000 2021
26152.0
Thu May 13 19:45:34 +0000 2021
11672.0
Thu May 13 19:53:45 +0000 2021
12163.0
Thu May 13 22:06:53 +0000 2021
20151.0
Thu May 13 21:21:17 +0000 2021
17415.0
Thu May 13 22:29:40 +0000 2021
21518.0
Thu May 13 16:45:58 +0000 2021
896.0
Thu May 13 16:50:52 +0000 2021
1190.0
Thu May 13 21:48:13 +0000 2021
19031.0
Thu May 13 16:31:41 +0000 2021
39.0
Thu May 13 16:36:03 +0000 2021
301.0
Fri May 14 01:18:07 +0000 2021
31625.0
Thu May 13 20:51:45 +0000 2021
15643.0
Fri May 14 01:41:54 +0000 2021
33052.0
Fri May 14 18:13:42 +0000 2021
92560.0
Thu May 13 22:01:40 +0000 2021
19838.0
Fri May 14 11:30:37 +0000 2021
68375.0
Thu May 13 18:41:30 +0000 2021
7828.0
Thu May 13 16

In [ ]:
len(db['followers'].find_one({'user_id': 18927441})['followers'])

In [ ]:
# db['followers'].remove({'user_id': 18927441})

In [ ]:
users= [18927441]

In [ ]:
for user in users:
    print(user, get_followers_len(user, db))

In [ ]:
for user in users:
    print(user, get_cascade_size(user, COLLECTION_NAME, db))

In [ ]:
# Check if the text is same
db[COLLECTION_NAME].distinct('retweet_count')

In [ ]:
db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')

In [ ]:
db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')

In [ ]:
# Check if the text is same
cache = db[COLLECTION_NAME].distinct('retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweet_count':x})['text'])
    print('*'*5)

In [ ]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.retweet_count':x})['id'])
    print('*'*5)

In [ ]:
cache = db[COLLECTION_NAME].distinct('retweeted_status.quoted_status.retweet_count')
print(cache[-5:])
for x in cache[-5:]:
    print(db[COLLECTION_NAME].find_one({'retweeted_status.quoted_status.retweet_count':x})['retweeted_status.quoted_status.id'])
    print('*'*5)

In [6]:
remove_duplicates(root)

0
0


In [7]:
for i in range(2,10):
    add_depth_nodes(root, i, tweet_user_ids, db, COLLECTION_NAME)
    remove_duplicates(root)

[Node('/18927441/1343243919790895115', created_at='Thu May 13 16:34:43 +0000 2021', difference_seconds=221.0), Node('/18927441/1099588868766978048', created_at='Thu May 13 20:25:25 +0000 2021', difference_seconds=14063.0), Node('/18927441/1254242876466311168', created_at='Thu May 13 21:23:48 +0000 2021', difference_seconds=17566.0), Node('/18927441/708591362', created_at='Thu May 13 18:09:55 +0000 2021', difference_seconds=5933.0), Node('/18927441/1325950008202563590', created_at='Fri May 14 16:13:03 +0000 2021', difference_seconds=85321.0)]
 user_id: 18927441, diff_s: 0
├──  user_id: 1343243919790895115, diff_s: 221.0
├──  user_id: 1099588868766978048, diff_s: 14063.0
├──  user_id: 1254242876466311168, diff_s: 17566.0
├──  user_id: 708591362, diff_s: 5933.0
├──  user_id: 1325950008202563590, diff_s: 85321.0
├──  user_id: 1347403143231930370, diff_s: 26152.0
├──  user_id: 728898445, diff_s: 11672.0
├──  user_id: 1364780663258415106, diff_s: 12163.0
├──  user_id: 1233959126046314496, di

In [8]:
# export tree
from anytree.exporter import JsonExporter, DictExporter
exporter = DictExporter()
data = exporter.export(root)
with open(COLLECTION_NAME + '_tree.json', 'w') as fp:
    json.dump(data, fp)   